## UNet Model

In [1]:
import os
from google.colab import drive
import torch
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import gc
from torch.utils.data import DataLoader, TensorDataset
drive.mount('/content/drive')

Mounted at /content/drive


### Load dataset

In [ ]:
def generate_npy_names(dir, n):
  """returns generated names of preprocessed dataset saved into npy files"""
  paths = []
  for i in range(n):
    if i == 0:
      continue
    paths.append(f'{dir}{i}.npy')
  return paths

training_path = '/content/drive/My Drive/MRI_segmentation_UNet/cropped_multi_channel_img_'
mask_path = '/content/drive/My Drive/MRI_segmentation_UNet/cropped_mask_'
val_path = '/content/drive/My Drive/MRI_segmentation_UNet/cropped_multi_channel_val_'
n = 8
training_dirs = generate_npy_names(training_path, n)
masks_dirs = generate_npy_names(mask_path, n)
val_dirs = generate_npy_names(val_path, 3)

In [ ]:
def load_batches(training_dir, masks_dir, batch_size):
  """args:
          training_dir (list)
          masks_dir (list)
          batch_size (int)
    returns:
          generator yields X, y (training portion) of a given batch_size for each iteration.
    it can be used as an alternative to pytorch's DataLoader()"""
  L = len(training_dir)
  i = 0
  while i < L:
    data = np.load(training_dir[i]).astype(np.float32)
    mask = np.load(masks_dir[i]).astype(np.float32)
    start = 0
    end = batch_size
    l = 0

    while l <= 11:

      yield data[start:end], mask[start:end]

      start+=batch_size
      end += batch_size
      l+=1
    i+=1

def load_data(dir):
  """ loads data as numpy array for training/validation from .npy files"""
  L = len(dir)
  i = 0
  x = np.array([])

  while i < L:
    data = np.load(dir[i]).astype(np.float32)
    x = np.append(x, data)
    i += 1
  return x


In [ ]:
X = load_data(training_dirs)
y = load_data(masks_dirs)
# X_val = load_data(val_dirs)

In [ ]:
X = X.reshape(368, 128, 128, 128, 3)
y = y.reshape(368, 128, 128, 128)
# X_val = X_val.reshape(125, 128, 128, 128, 3)

In [ ]:
X.shape, y.shape #, X_val.shape

((368, 128, 128, 128, 3), (368, 128, 128, 128))

In [ ]:
int(368 * 0.2) + 250

323

In [ ]:
def train_val_split(X, y, train_size = 0.6, plot = True):
  """ splits training portion into train & val  """
  test_val_size = (1 - train_size) // 2 # split test and val portion equal percentages
  train_idx = int(X.shape[0] * train_size)
  val_idx = train_idx + test_val_size

  train_x = X[0:train_idx, :, :, :, :]
  train_y = y[0:train_idx, :, :, :]

  test_x = X[train_idx:val_idx, :, :, :, :]
  test_y = y[val_idx:, :, :, :]

  if plot:


  # return train_x, train_y, test_x, test_y

In [ ]:
X_train, y_train, X_val, y_val = train_val_split(X, y)

In [ ]:
train_dataset = TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).float())
train_load = DataLoader(train_dataset, batch_size = 16, shuffle = True)

In [ ]:
val_dataset = TensorDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).float())
val_load = DataLoader(val_dataset, batch_size = 16, shuffle = True)

In [ ]:
# val_dataset = TensorDataset(torch.from_numpy(X_val).float())
# val_load = DataLoader(val_dataset, batch_size = 16, shuffle = True)

In [ ]:
class Encoder(nn.Module):
  def __init__(self, in_channels, out_channels, max_pool = True):

    self.in_channels = in_channels
    self.out_channels = out_channels

    super().__init__()
    self.encoder1 = nn.Sequential(
        nn.Conv3d(in_channels, out_channels, kernel_size = 3, padding = 1),
        nn.ReLU(inplace = True),

        nn.Conv3d(out_channels, out_channels, kernel_size = 3, padding = 1),
        nn.ReLU(inplace = True)
    )

    self.maxpool = nn.MaxPool3d(kernel_size = 2, stride = 2)

  def forward(self, x):
   x = self.encoder1(x)
   p = self.maxpool(x)
   return x, p

# upsampling
class Decoder(nn.Module):
  def __init__(self, in_channels, out_channels):

    self.in_channels = in_channels
    self.out_channels = out_channels
    super().__init__()

                                                 #inchannels // 2
    # self.upconv = nn.ConvTranspose3d(in_channels, in_channels, kernel_size=3, stride = 2

    self.double_conv = nn.Sequential(
        nn.Conv3d(in_channels, out_channels, kernel_size = 3, padding = 1),
        nn.ReLU(inplace = True),

        nn.Conv3d(out_channels, out_channels, kernel_size = 3, padding = 1),
        nn.ReLU(inplace = True)
    )

                       #p
  def forward(self, x1, x2):

    #perform interpolation as an alternative to transpose3d
    x1 = F.interpolate(x1, size=x2.shape[2:], mode='trilinear', align_corners=False)
    #print(x1.shape, x2.shape)
    x = torch.cat([x1, x2], dim = 1)
    x = self.double_conv(x)
    return x


class UNet(nn.Module):
  def __init__(self, in_channels, out_channels):

    super().__init__()
    #downsample:
    self.encoder1 = Encoder(in_channels, 64)
    self.encoder2 = Encoder(64, 128)
    self.encoder3 = Encoder(128, 256)
    self.encoder4 = Encoder(256, 512)

    #bottle neck:
    self.convolution = nn.Sequential(
        nn.Conv3d(512, 1024, kernel_size = 3, padding = 1),
        nn.ReLU(inplace = True),

        nn.Conv3d(1024, 1024, kernel_size = 3, padding = 1),
        nn.ReLU(inplace = True)
    )

    self.decoder1 = Decoder(1024 + 512, 512)
    self.decoder2 = Decoder(512 + 256, 256)
    self.decoder3 = Decoder(256 + 128, 128)
    self.decoder4 = Decoder(128 + 64, 64)

    self.output = nn.Conv3d(64, out_channels, kernel_size = 1)

  def forward(self, x):

    x1, p1 = self.encoder1(x)
    # print("x1: ", x1.shape, "p1: ", p1.shape)

    x2, p2 = self.encoder2(p1)
    # print("x2: ", x2.shape, "p2: ", p2.shape)

    x3, p3 = self.encoder3(p2)
    # print("x3: ", x3.shape, "p3: ", p3.shape)

    x4, p4 = self.encoder4(p3)
    # print("x4: ", x4.shape, "p4: ", p4.shape)

    bn = self.convolution(p4)
    # print("bn shape: ", bn.shape, "x3 shape: ", x4.shape)

    d1 = self.decoder1(bn, x4) # error in this line

    d2 = self.decoder2(d1, x3)
    # print("d1: ", d1.shape, "x3: ", x3.shape)

    d3 = self.decoder3(d2, x2)
    # print("problem test 7")
    d4 = self.decoder4(d3, x1)

    # print('output')
    with torch.no_grad():
      out = self.output(d4)
      return out
    cuda.torch.empty_cache()


In [ ]:
def avg(L):
  """returns average of a list"""
  return round(sum(L) / (len(L) + 0.0001), 2)

def class_distribution(training_dataset):
  """ takes: mask (tensor)
      returns: class distribution in a mask (y), used for dice loss """
  classes = [0, 1, 2, 3]
  num_classes = len(classes)
  w0s, w1s, w2s, w3s = [], [], [], []
  for x, y in training_dataset:
    for mask in y:
      val, ct = np.unique(y, return_counts = True)
      # print(ct)
      c = dict(zip(val, ct))
      total = sum(c.values())
      w0 = round(total / (num_classes * ct[0]), 2)
      w1 = round(total / (num_classes * ct[1]), 2)
      w2 = round(total / (num_classes * ct[2]), 2)
      if len(ct) == 4:
        w3 = round(total / (num_classes * ct[3]), 2)
        w3s.append(w3)
      w0s.append(w0)
      w1s.append(w1)
      w2s.append(w2)

  weights = [avg(w0s), avg(w1s), avg(w2s), avg(w3s)]
  norm_weights = [round(w / sum(weights), 4) for w in weights]

  return norm_weights

In [ ]:
# def ohe(a):
#   """returns numpy array one hot encoded"""
#   num_classes = np.unique(a).shape[0]
#   ohe = np.zeros((a.shape[0], a.shape[1], a.shape[2], num_classes))

#   ohe[:, :, :, 0] = (a == 0)
#   ohe[:, :, :, 1] = (a == 1)
#   ohe[:, :, :, 2] = (a == 2)
#   ohe[:, :, :, 3] = (a == 3)
#   return ohe

In [ ]:
def dice_loss(pred, target, weights, smooth = 1):
  """returns scalar dice"""

  classes = pred.shape[1]
  target = F.one_hot(target.long(), classes)
  # print("pred: ", pred.shape, "target: ", target.shape)
  target = target.permute(0, 4, 1, 2, 3)
  # print("pred: ", pred.shape, "target: ", target.shape)
  pred = F.softmax(pred, dim = 1)

  coefficient =  0

  for i in range(classes):

    pred_i = pred[:, i, :, :, :]
    target_i = target[:, i, :, :]

    intersection = (pred_i * target_i).sum(dim = (1, 2, 3))
    u = (pred_i + target_i).sum(dim = (1, 2, 3))

    coefficient += (2 * intersection + smooth) / (u + smooth)
    dice = weights[i] * ( 1 - coefficient)


  return dice.mean()

## Training


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
NUM_EPOCHS = 50
SEED = torch.manual_seed(42)
lr = 3e-1
model = UNet(in_channels = 3, out_channels = 4).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr = lr)
preds = []

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
weights = class_distribution(train_load)

In [ ]:
weights

[np.float64(0.003), np.float64(0.4504), np.float64(0.1296), np.float64(0.417)]

In [ ]:
for epoch in tqdm(range(NUM_EPOCHS)):

  train_loss = 0.0
  print("epoch ", epoch)
  for X_train, y_train in train_load:

    X_train = X_train.permute(0, 4, 1, 2, 3)
    X_train = X_train.to(device)
    y_train = y_train.to(device)

    model.train()
    optimizer.zero_grad()
    y_pred = model(X_train)
    preds.append(y_pred)

    loss = dice_loss(y_pred, y_train, weights)
    train_loss += loss.item()
    loss.requires_grad = True
    loss.backward()
    optimizer.step()
    # print("loss: ", loss)

  train_loss /= len(train_load)
  print('train loss: ', train_loss)

  0%|          | 0/50 [00:00<?, ?it/s]

epoch  0
train loss:  0.22697110538897308
epoch  1


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 GiB. GPU 0 has a total capacity of 39.56 GiB of which 1.78 GiB is free. Process 29624 has 37.76 GiB memory in use. Of the allocated memory 33.83 GiB is allocated by PyTorch, and 3.45 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)